In [26]:
pip install -U langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.3/389.3 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.2.4
    Uninstalling langchain-openai-0.2.4:
      Successfully uninstalled langchain-openai-0.2.4
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully unin

In [28]:
LANGCHAIN_TRACING_V2=True
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY="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"
LANGCHAIN_PROJECT="DF_europe_agent_demo"

In [84]:
#system imports
import os
import json
from dotenv import load_dotenv
import re
import requests
load_dotenv('/Users/joe.constantino/Desktop/playground/tableau_langchain/.env')

#langchain imports
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from langchain.agents import initialize_agent, AgentType


#tableau_langchain imports
from hbiQuery import get_data
from readMetadata import read
from setPrompt import instantiate_prompt, get_values
from prompts.nlq_to_vds import prompt

In [87]:
datasource = os.getenv('DATASOURCE_LUID')
read_metadata_url = os.getenv('READ_METADATA')
query_datasource = os.getenv('QUERY_DATASOURCE')
token = os.getenv('AUTH_TOKEN')

In [88]:
metadata = read(read_url=read_metadata_url, datasource_luid=datasource, auth_secret=token)
prompt = instantiate_prompt(metadata = metadata)

reading in your field metadata...
looking up field values...
prompt is ready!


In [93]:
@tool
def get_data(query: str) -> dict:
    """
    Generates the VizQL data service query payload for retrieving relevant data from a Tableau published datasource.
    It then passes the payload to the query_datasource endpoint exposed by the VizQL data service and returns a json 
    formatted dataframe as output. 

    Args:
        query (str): A natural language query describing the data to retrieve.

    Returns:
        dict: A JSON payload suitable for the VizQL data service.
    """
    llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    prompt_template = ChatPromptTemplate.from_messages([
      SystemMessage(content=json.dumps(prompt)),
      ("user", "{utterance}")])
    
    output_parser = StrOutputParser()
    
    chain = prompt_template | llm | output_parser
    
    output = chain.invoke(query)
    parsed_output = output.split('JSON_payload')[1]
    match = re.search(r'{.*}', parsed_output, re.DOTALL)
    if match:
        json_string = match.group(0)
    payload = json.loads(json_string)

    query_url = os.getenv('QUERY_DATASOURCE')
    datasource_luid = os.getenv('DATASOURCE_LUID')
    auth_secret = os.getenv('AUTH_TOKEN')

    payload = json.dumps({
        "datasource": {
            "datasourceLuid": datasource_luid
        },
        "query": payload
    })

    headers = {
        'X-Tableau-Auth': auth_secret,
        'Content-Type': 'application/json'
    }

    response = requests.post(query_url, headers=headers, data=payload)

    if response.status_code == 200:
        data = response.json().get('data')
        return data
    else:
        raise Exception(f"Failed to fetch data from the API. Status code: {response.status_code}, Response: {response.text}")


In [92]:
get_data.invoke("what are sales by state?")

[{'State/Province': 'Florida', 'SUM(Sales)': 89473.71},
 {'State/Province': 'Oregon', 'SUM(Sales)': 17431.15},
 {'State/Province': 'New Hampshire', 'SUM(Sales)': 7292.52},
 {'State/Province': 'New Brunswick', 'SUM(Sales)': 225.7},
 {'State/Province': 'Minnesota', 'SUM(Sales)': 29863.15},
 {'State/Province': 'North Dakota', 'SUM(Sales)': 919.91},
 {'State/Province': 'Newfoundland and Labrador', 'SUM(Sales)': 515.2},
 {'State/Province': 'Arizona', 'SUM(Sales)': 35282.0},
 {'State/Province': 'Iowa', 'SUM(Sales)': 4579.76},
 {'State/Province': 'Rhode Island', 'SUM(Sales)': 22627.96},
 {'State/Province': 'Arkansas', 'SUM(Sales)': 11678.13},
 {'State/Province': 'Ontario', 'SUM(Sales)': 7009.01},
 {'State/Province': 'Manitoba', 'SUM(Sales)': 1930.78},
 {'State/Province': 'Texas', 'SUM(Sales)': 170188.05},
 {'State/Province': 'Mississippi', 'SUM(Sales)': 10771.34},
 {'State/Province': 'Nevada', 'SUM(Sales)': 16729.1},
 {'State/Province': 'Colorado', 'SUM(Sales)': 32108.12},
 {'State/Province':

In [95]:
# Initialize the LLM and the agent with the tool
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
tools = [get_data]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,  # Use OpenAI's function calling
    verbose=True
)

# Run the agent
response = agent.run("how do sales compare to profits for each segment?")
print(response)



> Entering new AgentExecutor chain...

Invoking: `get_data` with `{'query': 'compare sales to profits for each segment'}`


[{'Segment': 'Consumer', 'SUM(Sales)': 1170659.79, 'SUM(Profit)': 136371.45}, {'Segment': 'Home Office', 'SUM(Sales)': 440068.43, 'SUM(Profit)': 61675.73}, {'Segment': 'Corporate', 'SUM(Sales)': 715806.13, 'SUM(Profit)': 94249.64}]Here is the comparison of sales to profits for each segment:

1. **Consumer Segment**
   - Total Sales: $1,170,659.79
   - Total Profit: $136,371.45

2. **Home Office Segment**
   - Total Sales: $440,068.43
   - Total Profit: $61,675.73

3. **Corporate Segment**
   - Total Sales: $715,806.13
   - Total Profit: $94,249.64

This data shows the total sales and profits for each segment, allowing for a comparison of their performance.

> Finished chain.
Here is the comparison of sales to profits for each segment:

1. **Consumer Segment**
   - Total Sales: $1,170,659.79
   - Total Profit: $136,371.45

2. **Home Office Segment**
   - Total Sa